In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from Vocab import *
from model import AttentionModel

import os
import string

In [2]:
#Hyperparams
lr = 1
embed_size = 256
hidden_size = 512

In [3]:
sentence = "It stands for Java Development Kit. It is the tool necessary to compile, document and package Java programs."

In [4]:
sentence = cleanText(sentence)

In [5]:
vocab = getVocab('text.txt')

In [6]:
vocab_size = vocab.size()

In [7]:
model = AttentionModel(embed_size, hidden_size, vocab_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

In [8]:
corr_sentence = sentence
corr_tensor = torch.Tensor(vocab.getSentenceArray(sentence))

In [9]:
test_sentences = ["It stands for Java Development Kit. It is the tool necessary to compile, document and package Java programs.",
                  "It stands for Java Development Kit",
                  "I don't know",
                  "it is a good traversal",
                  "And its a goal!!!",
                  "It was developed by Java"]
test_sentences = [cleanText(s) for s in test_sentences]
test_tensors = [torch.tensor(vocab.getSentenceArray(s)) for s in test_sentences]
test_grades = [1, 0.7, 0.2, 0.1, 0, 0.4]

In [10]:
for e in range(50):
    total_loss = 0
    for t in range(len(test_tensors)):
        optimizer.zero_grad()
        
        pred = model(corr_tensor.long(), test_tensors[t].long()).view(1)
        y = torch.tensor(test_grades[t]).float().view(1)
        loss = criterion(pred, y)
        total_loss += loss.item()
    
        loss.backward()
        optimizer.step()
    print("Epoch:", e, "Loss:", total_loss)

/home/krypt/myStuff/pytorch/tuts/1/AnswerRatingPredictor/model.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  corr_attn_params = F.softmax(self.lin_attn(corr).view(1, -1))
/home/krypt/myStuff/pytorch/tuts/1/AnswerRatingPredictor/model.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  test_attn_params = F.softmax(self.lin_attn(test).view(1, -1))
/home/krypt/.local/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 0 Loss: 0.7908607171848416
Epoch: 1 Loss: 0.654195997864008
Epoch: 2 Loss: 0.4944455726072192
Epoch: 3 Loss: 0.2983965618768707
Epoch: 4 Loss: 0.26732729584909976
Epoch: 5 Loss: 0.20855257962830365
Epoch: 6 Loss: 0.181807820099948
Epoch: 7 Loss: 0.18797262955922633
Epoch: 8 Loss: 0.11003358568996191
Epoch: 9 Loss: 0.15872628315082693
Epoch: 10 Loss: 0.11339493340346962
Epoch: 11 Loss: 0.17005356433219276
Epoch: 12 Loss: 0.08018149917097617
Epoch: 13 Loss: 0.08818949795931985
Epoch: 14 Loss: 0.0795739506429527
Epoch: 15 Loss: 0.08080024803430774
Epoch: 16 Loss: 0.08371634903596714
Epoch: 17 Loss: 0.13056559237884358
Epoch: 18 Loss: 0.08196555566973984
Epoch: 19 Loss: 0.07044181019682583
Epoch: 20 Loss: 0.06975771398720099
Epoch: 21 Loss: 0.05973584286402911
Epoch: 22 Loss: 0.08269635564647615
Epoch: 23 Loss: 0.057123692822642624
Epoch: 24 Loss: 0.06599061568704201
Epoch: 25 Loss: 0.0449129528242338
Epoch: 26 Loss: 0.05380491996766068
Epoch: 27 Loss: 0.03405590419424698
Epoch: 28 

In [11]:
model(corr_tensor.long(), test_tensors[5].long())

/home/krypt/myStuff/pytorch/tuts/1/AnswerRatingPredictor/model.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  corr_attn_params = F.softmax(self.lin_attn(corr).view(1, -1))
/home/krypt/myStuff/pytorch/tuts/1/AnswerRatingPredictor/model.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  test_attn_params = F.softmax(self.lin_attn(test).view(1, -1))
/home/krypt/.local/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[0.4436]], grad_fn=<SigmoidBackward>)

In [12]:
model(corr_tensor.long(), torch.tensor(vocab.getSentenceArray("Its nice")).long())

/home/krypt/myStuff/pytorch/tuts/1/AnswerRatingPredictor/model.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  corr_attn_params = F.softmax(self.lin_attn(corr).view(1, -1))
/home/krypt/myStuff/pytorch/tuts/1/AnswerRatingPredictor/model.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  test_attn_params = F.softmax(self.lin_attn(test).view(1, -1))
/home/krypt/.local/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[0.0826]], grad_fn=<SigmoidBackward>)

In [14]:
torch.save(model, "saved_models/model1.pt")